In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [4]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(39010, 2831) (12128, 2831)


In [5]:
features_list =[
    
#Bureau Intutive features
'Num_Enq_all_loans_last_9M_3M',
'Utilization_all',
'total_emi',
'Utilization_CREDIT CARD_opened_last_36M',
'Utilization_PERSONAL LOAN_opened_last_9M',
'num_CREDIT CARD_enq_last_6M',
'num_CONSUMER LOAN_enq_last_36M',
'Num_of_PERSONAL LOAN_opened_last6M_12M',
'highest_sanction_amount_credit_cards',
'num_CONSUMER LOAN_enq_last_6M',
'num_PL_LE_1_Lac_opened_in_last_12M',
'Total_Sanction_amt_PERSONAL LOAN_Opened_last_24M',
'Age_of_oldest_open_account',
    
#Banking Intutive features

'num_credit_gt_salary',    
'upi_trans_count',
'ratio_balance_after_7_days_max_credit_M3',    
'ratio_max_credit_to_total_credit_M2',
'spend_vs_salary_ratio',
'bounced_transactions_count',
'ratio_total_debit_to_max_credit_M2_M3',
'min_balance_last_3_months',
'min_balance_M2',
'days_balance_lt_5000_M3',
'days_balance_lt_10000_M3',
'Max_Credit_Avg_M1_M2_M3',
'Max_Credit_Diff_M1_M2_M3',
'amount_debits_last_2_months',
'sum_insurance_debit',
'avg_num_debits_last_2_months',
'upi_trans_debit_sum',
'cash_withdrawal_sum',
    
#new banking features
'diff_btw_stated_salary_vs_actual_salary',
'balance_6th_day_M2',
'ratio_6th_day_balance_to_max_credit_M3',

#Loan application Features

'deviation_from_median_income/standard_deviation_income#last_365_Days',
'loan_amount_to_income#last_365_Days',

    
#ID Features
'dm206s',
'g406s',


# # Reconcilated Features
'percentage_ever_delinquent', 
'months_since_most_recent_delinquency_installment', 
'percentage_utilization_gt_75',
'Num_of_installment_loan_opened_last_6M',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'month_year',
'90_in_24mob'
              
]




In [6]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3022536/282645959.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3022536/282645959.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [8]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23406, 44), (15604, 44), (23406,), (15604,))

In [9]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(18724, 44) (4682, 44) (15604, 44)


In [10]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [11]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,

    'subsample': 0.4,
    'colsample_bytree': 0.4,
     'min_child_weight':25,

    'lambda': 5.5,
    'alpha':5.0,
    'gamma':4.5,

    'scale_pos_weight':3.8,
    'random_state': 24
    
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 600,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.61483	eval-auc:0.62419
[10]	train-auc:0.64236	eval-auc:0.64840
[20]	train-auc:0.64661	eval-auc:0.65276
[30]	train-auc:0.64978	eval-auc:0.65566
[40]	train-auc:0.65170	eval-auc:0.65408
[50]	train-auc:0.65404	eval-auc:0.65565
[60]	train-auc:0.65542	eval-auc:0.65607
[70]	train-auc:0.65673	eval-auc:0.65638
[80]	train-auc:0.65842	eval-auc:0.65663
[90]	train-auc:0.65896	eval-auc:0.65744
[100]	train-auc:0.65960	eval-auc:0.65743
[110]	train-auc:0.66051	eval-auc:0.65749
[120]	train-auc:0.66147	eval-auc:0.65787
[130]	train-auc:0.66304	eval-auc:0.65897
[140]	train-auc:0.66486	eval-auc:0.66004
[150]	train-auc:0.66562	eval-auc:0.66072
[160]	train-auc:0.66633	eval-auc:0.66138
[170]	train-auc:0.66711	eval-auc:0.66187
[180]	train-auc:0.66783	eval-auc:0.66215
[190]	train-auc:0.66866	eval-auc:0.66255
[200]	train-auc:0.66952	eval-auc:0.66332
[210]	train-auc:0.67037	eval-auc:0.66327
[220]	train-auc:0.67127	eval-auc:0.66356
[230]	train-auc:0.67181	eval-auc:0.66371
[240]	train-auc:0.67248	eva

In [12]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.69, Gini: 0.38, KS: 0.27
Eval AUC: 0.67, Gini: 0.34, KS: 0.26
Test AUC: 0.67, Gini: 0.34, KS: 0.25
OOT AUC: 0.68, Gini: 0.36, KS: 0.27


In [13]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.397224         0.645057        0.828032        0.580391   1873   
9          0.259615         0.543888        0.580387        0.513406   1872   
8          0.205662         0.487239        0.513400        0.462682   1872   
7          0.179391         0.442321        0.462651        0.423341   1873   
6          0.167201         0.406399        0.423322        0.389879   1872   
5          0.142094         0.375243        0.389840        0.361570   1872   
4          0.121196         0.348697        0.361569        0.336059   1873   
3          0.098825         0.323041        0.336054        0.309463   1872   
2          0.067842         0.295425        0.309459        0.279292   1872   
1          0.043246         0.249858        0.279253        0.160969   1873   

        label_good  label_bad  
Decile                         
10            1129        744  
9             1386        486  
8             1487        385  
7             1537        336  
6             1559        313  
5             1606        266  
4             1646        227  
3             1687        185  
2             1745        127  
1             1792         81

In [14]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.368994         0.644317        0.819143        0.579631   1561   
9          0.280128         0.542316        0.579500        0.509931   1560   
8          0.210897         0.484453        0.509919        0.461048   1560   
7          0.176169         0.441460        0.461027        0.422576   1561   
6          0.158974         0.405581        0.422570        0.389254   1560   
5          0.136538         0.374611        0.389253        0.360817   1560   
4          0.119795         0.348496        0.360813        0.336419   1561   
3          0.102564         0.323344        0.336408        0.309994   1560   
2          0.096154         0.295499        0.309957        0.278374   1560   
1          0.058937         0.249298        0.278340        0.170948   1561   

        label_good  label_bad  
Decile                         
10             985        576  
9             1123        437  
8             1231        329  
7             1286        275  
6             1312        248  
5             1347        213  
4             1374        187  
3             1400        160  
2             1410        150  
1             1469         92

In [15]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.368508         0.642056        0.825574        0.578174   1213   
9          0.272053         0.540897        0.578163        0.509778   1213   
8          0.215993         0.485720        0.509760        0.463249   1213   
7          0.212871         0.442178        0.463204        0.422625   1212   
6          0.150041         0.405243        0.422590        0.388932   1213   
5          0.144270         0.374350        0.388874        0.360202   1213   
4          0.126238         0.348521        0.360195        0.336889   1212   
3          0.095631         0.324183        0.336877        0.311101   1213   
2          0.081616         0.297840        0.311059        0.283184   1213   
1          0.055235         0.253022        0.283135        0.175746   1213   

        label_good  label_bad  
Decile                         
10             766        447  
9              883        330  
8              951        262  
7              954        258  
6             1031        182  
5             1038        175  
4             1059        153  
3             1097        116  
2             1114         99  
1             1146         67

In [16]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
37                                              g406s       162.0
8                highest_sanction_amount_credit_cards       134.0
12                         Age_of_oldest_open_account       122.0
30                                cash_withdrawal_sum        91.0
0                        Num_Enq_all_loans_last_9M_3M        82.0
43                          Num_Enq_non_BANK_last_12M        78.0
1                                     Utilization_all        74.0
4            Utilization_PERSONAL LOAN_opened_last_9M        67.0
38                         percentage_ever_delinquent        60.0
34  deviation_from_median_income/standard_deviatio...        58.0
3             Utilization_CREDIT CARD_opened_last_36M        57.0
42                           No of enq in past 90days        55.0
18                         bounced_transactions_count        50.0
39   months_since_most_recent_delinquency_installment        44.0
9         

In [17]:
feature_importance_df

Feature  Importance
37                                              g406s       162.0
8                highest_sanction_amount_credit_cards       134.0
12                         Age_of_oldest_open_account       122.0
30                                cash_withdrawal_sum        91.0
0                        Num_Enq_all_loans_last_9M_3M        82.0
43                          Num_Enq_non_BANK_last_12M        78.0
1                                     Utilization_all        74.0
4            Utilization_PERSONAL LOAN_opened_last_9M        67.0
38                         percentage_ever_delinquent        60.0
34  deviation_from_median_income/standard_deviatio...        58.0
3             Utilization_CREDIT CARD_opened_last_36M        57.0
42                           No of enq in past 90days        55.0
18                         bounced_transactions_count        50.0
39   months_since_most_recent_delinquency_installment        44.0
9                       num_CONSUMER LOAN_enq_last_6M        44.0
5                         num_CREDIT CARD_enq_last_6M        43.0
14                                    upi_trans_count        42.0
7              Num_of_PERSONAL LOAN_opened_last6M_12M        39.0
22                            days_balance_lt_5000_M3        35.0
21                                     min_balance_M2        34.0
41             Num_of_installment_loan_opened_last_6M        27.0
35                loan_amount_to_income#last_365_Days        26.0
33             ratio_6th_day_balance_to_max_credit_M3        24.0
20                          min_balance_last_3_months        23.0
15           ratio_balance_after_7_days_max_credit_M3        23.0
32                                 balance_6th_day_M2        22.0
6                      num_CONSUMER LOAN_enq_last_36M        22.0
2                                           total_emi        22.0
13                               num_credit_gt_salary        21.0
27                                sum_insurance_debit        21.0
23                           days_balance_lt_10000_M3        19.0
10                 num_PL_LE_1_Lac_opened_in_last_12M        19.0
40                       percentage_utilization_gt_75        18.0
36                                             dm206s        16.0
28                       avg_num_debits_last_2_months        16.0
16                ratio_max_credit_to_total_credit_M2        16.0
11   Total_Sanction_amt_PERSONAL LOAN_Opened_last_24M        15.0
17                              spend_vs_salary_ratio        14.0
29                                upi_trans_debit_sum        11.0
25                           Max_Credit_Diff_M1_M2_M3         9.0
19              ratio_total_debit_to_max_credit_M2_M3         7.0
24                            Max_Credit_Avg_M1_M2_M3         6.0
26                        amount_debits_last_2_months         6.0
31            diff_btw_stated_salary_vs_actual_salary         6.0

In [18]:
# Save the pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BBILA_Train_Pred_9024.csv',index=False)
Eval.to_csv('LeoPayu_BBILA_Eval_Pred_9024.csv',index=False)
Test.to_csv('LeoPayu_BBILA_Test_Pred_9024.csv',index=False)
Oot.to_csv('LeoPayu_BBILA_Oot_Pred_9024.csv',index=False)


In [19]:
#Save the model
xgb_model.save_model('LeoPayu_BBILA_90_in_24mob.json')
print('Model Saved')

Model Saved
